# daewoo_module.py

In [1]:
import pandas as pd
import os
import tensorflow as tf
from daewoo_module import *
import sklearn.metrics as skm
import time
from collections import Counter

C:\Users\Jay\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
NUM_CLASSES = 3

In [38]:
def set_input(img, label):
    np.random.seed(1234)
    idx = np.random.permutation(len(img))
    tr_idx = idx[:round(0.8 * len(idx))]
    ts_idx = idx[round(0.8 * len(idx)):]

    train_img = img[tr_idx]
    train_label = label[tr_idx]
    test_img = img[ts_idx]
    test_label = label[ts_idx]

    train_img_tensor = tf.constant(train_img)
    train_label_tensor = tf.constant(train_label)
    test_img_tensor = tf.constant(test_img)
    test_label_tensor = tf.constant(test_label)

    return train_img_tensor, train_label_tensor, test_img_tensor, test_label_tensor, tr_idx, ts_idx

In [4]:
# string 텐서를 img 텐서로 변환 후 crop
def input_tensor(img_path, label):
    label_crop = tf.one_hot(label, NUM_CLASSES)
    
    img_file = tf.read_file(img_path)
    img_decoded = tf.image.decode_png(img_file)
    img_crop = tf.image.crop_to_bounding_box(img_decoded, 135, 0, 135, 135)
    img_float = tf.to_float(img_crop)
    img_crop = tf.random_crop(img_float, size=[135, 135, 3])
    
    for i in range(1,10):
        _label = tf.one_hot(label, NUM_CLASSES)
        
        _img_crop = tf.image.crop_to_bounding_box(img_decoded, 135, 38*i, 135, 135)
        _img_float = tf.to_float(_img_crop)
        _img_crop = tf.random_crop(_img_float, size=[135, 135, 3])
        
        label_crop = tf.concat([label_crop, _label], axis=0)
        img_crop = tf.concat([img_crop, _img_crop], axis=0)

    return tf.reshape(img_crop, [-1,135,135,3]), tf.reshape(label_crop, [-1,3])

In [5]:
def make_batch(dataset):
    dataset_0 = dataset.filter(lambda x,y: tf.reshape(tf.equal(tf.argmax(y), tf.argmax(tf.constant([1,0,0], tf.float32))), []))
    dataset_1 = dataset.filter(lambda x,y: tf.reshape(tf.equal(tf.argmax(y), tf.argmax(tf.constant([0,1,0], tf.float32))), [])).repeat()
    dataset_2 = dataset.filter(lambda x,y: tf.reshape(tf.equal(tf.argmax(y), tf.argmax(tf.constant([0,0,1], tf.float32))), [])).repeat()
    
    datasets = tf.data.Dataset.zip((dataset_0, dataset_1, dataset_2))
    datasets = datasets.flat_map(lambda ex_0, ex_1, ex_2: tf.data.Dataset.from_tensors(ex_0).concatenate(tf.data.Dataset.from_tensors(ex_1))
                                 .concatenate(tf.data.Dataset.from_tensors(ex_2)))
    
    return datasets

In [6]:
def conv2d(x, num_outputs, batch_norm=True):
    if batch_norm is True:
        conv_bn = tf.contrib.layers.batch_norm
    else:
        conv_bn = None

    conv = tf.contrib.layers.conv2d(inputs=x,
                                    num_outputs=num_outputs,
                                    kernel_size=(3, 3),
                                    normalizer_fn=conv_bn,
                                    activation_fn=tf.nn.relu)
    return conv

In [7]:
def pooling(x):
    pool = tf.contrib.layers.max_pool2d(inputs=x, kernel_size=(2, 2))
    return pool

In [8]:
def dense(x, output, fn=tf.nn.relu, batch_norm=True):
    if batch_norm is True:
        fc_bn = tf.contrib.layers.batch_norm
    else:
        fc_bn = None
    fc = tf.contrib.layers.fully_connected(inputs=x,
                                           num_outputs=output,
                                           normalizer_fn=fc_bn,
                                           activation_fn=fn)
    return fc

In [9]:
class VGG16():
    def __init__(self, x, y, bn, classification):
        
        with tf.name_scope("input"):
            self.x = x
            self.y = y

        with tf.name_scope("layer_1"):
            conv1 = conv2d(x, 64, batch_norm=bn)
            conv2 = conv2d(conv1, 64, batch_norm=bn)
            pool1 = pooling(conv2)

        with tf.name_scope("layer_2"):
            conv3 = conv2d(pool1, 128, batch_norm=bn)
            conv4 = conv2d(conv3, 128, batch_norm=bn)
            pool2 = pooling(conv4)

        with tf.name_scope("layer_3"):
            conv5 = conv2d(pool2, 256, batch_norm=bn)
            conv6 = conv2d(conv5, 256, batch_norm=bn)
            conv7 = conv2d(conv6, 256, batch_norm=bn)
            pool3 = pooling(conv7)

        with tf.name_scope("layer_4"):
            conv8 = conv2d(pool3, 512, batch_norm=bn)
            conv9 = conv2d(conv8, 512, batch_norm=bn)
            conv10 = conv2d(conv9, 512, batch_norm=bn)
            pool4 = pooling(conv10)

        with tf.name_scope("layer_5"):
            conv11 = conv2d(pool4, 512, batch_norm=bn)
            conv12 = conv2d(conv11, 512, batch_norm=bn)
            conv13 = conv2d(conv12, 512, batch_norm=bn)
            pool5 = pooling(conv13)

        with tf.name_scope("FC_layer"):
            fc1 = tf.layers.flatten(pool5)
            fc2 = dense(fc1, 4096, batch_norm=bn)
            fc3 = dense(fc2, 4096, batch_norm=bn)

        self.learning_rate = tf.placeholder(tf.float32)
        self.global_step = tf.Variable(0, trainable=False, name='global_step')

        if classification is True:
            self.logits = dense(fc3, NUM_CLASSES, fn=None, batch_norm=True)
            self.loss = tf.losses.softmax_cross_entropy(onehot_labels=self.y, logits=self.logits)
            self.lr_decay = tf.train.exponential_decay(self.learning_rate, self.global_step, 1000, 0.9, staircase=True)
            self.extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            
            with tf.control_dependencies(self.extra_update_ops):
                self.adam = tf.train.AdamOptimizer(self.lr_decay).minimize(self.loss,
                                                                           global_step=self.global_step)
                self.sgd = tf.train.GradientDescentOptimizer(self.lr_decay).minimize(self.loss,
                                                                                     global_step=self.global_step)
                self.rms = tf.train.RMSPropOptimizer(self.lr_decay).minimize(self.loss,
                                                                             global_step=self.global_step)
                self.momentum = tf.train.MomentumOptimizer(self.lr_decay, momentum=0.9).minimize(self.loss,
                                                                                                 global_step=self.global_step)

            self.y_prob = tf.nn.softmax(self.logits)
            self.y_pred = tf.argmax(self.y_prob, 1)

            self.correct_prediction = tf.equal(self.y_pred, tf.arg_max(y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

            tf.summary.scalar("accuray", self.accuracy)
            tf.summary.scalar("loss", self.loss)

        else:
            self.logits = tf.layers.dense(fc3, 1, activation=tf.nn.relu)
            self.loss = tf.losses.mean_squared_error(labels=self.y, predictions=self.logits)
            self.lr_decay = tf.train.exponential_decay(self.learning_rate, self.global_step, 1000, 0.9, staircase=True)
            self.extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            
            with tf.control_dependencies(self.extra_update_ops):
                self.adam = tf.train.AdamOptimizer(self.lr_decay).minimize(self.loss,
                                                                           global_step=self.global_step)
                self.sgd = tf.train.GradientDescentOptimizer(self.lr_decay).minimize(self.loss,
                                                                                     global_step=self.global_step)
                self.rms = tf.train.RMSPropOptimizer(self.lr_decay).minimize(self.loss,
                                                                             global_step=self.global_step)
                self.momentum = tf.train.MomentumOptimizer(self.lr_decay, momentum=0.9).minimize(self.loss,
                                                                                                 global_step=self.global_step)
            
            tf.summary.scalar("loss", self.loss)

        self.merged_summary_op = tf.summary.merge_all()

# daewoo_train_original.py

In [10]:
os.environ['FOR_DISABLE_CONSOLE_CTRL_HANDLER'] = '1'

In [11]:
root_dir = "./input_data"
img_dir = "./input_data/figure/"
logs_path = os.path.join(root_dir, "graph")

In [12]:
img = np.array([img_dir + x for x in os.listdir(img_dir)])
label = pd.read_csv(os.path.join(root_dir, 'description.csv'), engine='python')

In [13]:
classification = True

batch_size = 64
epochs = 2

In [14]:
if classification is True:
    label = pd.cut(label['WVHT ft.y'], bins=[0, 5.2, 7.9, 100], labels=[0, 1, 2], include_lowest=True)
    label = np.array(label)
else:
    label = label['WVHT ft.y'].values
    label = ((label - np.mean(label)) / np.std(label)).reshape(-1, 1)

In [39]:
train_img_tensor, train_label_tensor, test_img_tensor, test_label_tensor, tr_idx, ts_idx = set_input(img, label)

In [16]:
train_imgs = tf.data.Dataset.from_tensor_slices((train_img_tensor, train_label_tensor))
test_imgs = tf.data.Dataset.from_tensor_slices((test_img_tensor, test_label_tensor))
infer_imgs = tf.data.Dataset.from_tensor_slices((test_img_tensor, test_label_tensor))

In [17]:
if classification is True:
    train_imgs = train_imgs.map(input_tensor).apply(tf.contrib.data.unbatch()).shuffle(buffer_size=100).apply(lambda x: make_batch(x)).batch(batch_size).repeat()
    test_imgs = test_imgs.map(input_tensor).apply(tf.contrib.data.unbatch()).shuffle(buffer_size=100).apply(lambda x: make_batch(x)).batch(batch_size).repeat()
    infer_imgs = infer_imgs.map(input_tensor).apply(tf.contrib.data.unbatch()).batch(batch_size)
else:
    train_imgs = train_imgs.map(input_tensor_regression).apply(tf.contrib.data.unbatch()).shuffle(buffer_size=100).apply(lambda x: make_batch(x)).batch(batch_size).repeat()
    test_imgs = test_imgs.map(input_tensor_regression).apply(tf.contrib.data.unbatch()).shuffle(buffer_size=100).apply(lambda x: make_batch(x)).batch(batch_size).repeat()
    infer_imgs = infer_imgs.map(input_tensor).apply(tf.contrib.data.unbatch()).batch(batch_size)

In [18]:
train_iterator = train_imgs.make_initializable_iterator()
test_iterator = test_imgs.make_initializable_iterator()
infer_iterator = infer_imgs.make_initializable_iterator()
handle = tf.placeholder(tf.string, shape=[])

In [19]:
iterator = tf.data.Iterator.from_string_handle(handle, train_imgs.output_types, train_imgs.output_shapes)
x, y = iterator.get_next()

In [20]:
# train class: [18204, 17525, 15748]
train_batches = 18204*3*10 // batch_size

In [21]:
model = VGG16(x, y, bn=True, classification=classification)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Instructions for updating:
Use `argmax` instead


In [22]:
if classification is True:
    model_name = "vgg16_classification_crop10"
else:
    model_name = "vgg16_regression_crop10"

In [23]:
start_time = time.time()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [24]:
sess = tf.Session(config=config)
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
train_handle = sess.run(train_iterator.string_handle())
test_handle = sess.run(test_iterator.string_handle())
infer_handle = sess.run(infer_iterator.string_handle())
train_writer = tf.summary.FileWriter(os.path.join(logs_path, model_name, 'train'), sess.graph)
test_writer = tf.summary.FileWriter(os.path.join(logs_path, model_name, 'test'))

In [25]:
LEARNING_RATE = 0.001
optimizer = model.rms

In [26]:
if classification is True:

    print("Training!")
    for i in range(epochs):
        print("-------{} Epoch--------".format(i + 1))
        sess.run(train_iterator.initializer)
        sess.run(test_iterator.initializer)
        for j in range(train_batches):
            summary, _, acc, loss_ = sess.run([model.merged_summary_op, optimizer, model.accuracy, model.loss],
                                              feed_dict={handle: train_handle, model.learning_rate: LEARNING_RATE})
            step = tf.train.global_step(sess, model.global_step)
            print("Training Iter : {}, Acc : {}, Loss : {:.4f}".format(step, acc, loss_))

            if j % 10 == 0:
                train_writer.add_summary(summary, step)
                summary, acc, loss_ = sess.run([model.merged_summary_op, model.accuracy, model.loss],
                                               feed_dict={handle: test_handle})
                print("Validation Iter : {}, Acc : {}, Loss : {:.4f}".format(step, acc, loss_))
                test_writer.add_summary(summary, step)

    print("-----------End of training-------------")

    end_time = time.time() - start_time
    print("{} seconds".format(end_time))

    saver.save(sess, os.path.join(logs_path, 'VGG16_classification_crop', model_name))

else:
    print("Training!")
    for i in range(epochs):
        print("-------{} Epoch--------".format(i + 1))
        sess.run(train_iterator.initializer)
        sess.run(test_iterator.initializer)
        for j in range(train_batches):
            summary, _, loss_ = sess.run([model.merged_summary_op, optimizer, model.loss],
                                         feed_dict={handle: train_handle, model.learning_rate: LEARNING_RATE})
            step = tf.train.global_step(sess, model.global_step)
            print("Training Iter : {}, Loss : {:.4f}".format(step, loss_))

            if j % 10 == 0:
                train_writer.add_summary(summary, step)
                summary, loss_ = sess.run([model.merged_summary_op, model.loss],
                                          feed_dict={handle: test_handle})
                print("Validation Iter : {}, Loss : {:.4f}".format(step, loss_))
                test_writer.add_summary(summary, step)

    print("-----------End of training-------------")

    end_time = time.time() - start_time
    print("{} seconds".format(end_time))

    saver.save(sess, os.path.join(logs_path, 'VGG16_regression_crop', model_name))

Training!
-------1 Epoch--------
Training Iter : 1, Acc : 0.265625, Loss : 1.3475
Validation Iter : 1, Acc : 0.34375, Loss : 1.4018
Training Iter : 2, Acc : 0.5, Loss : 1.0482
Training Iter : 3, Acc : 0.46875, Loss : 1.2122
Training Iter : 4, Acc : 0.5, Loss : 0.9832
Training Iter : 5, Acc : 0.59375, Loss : 1.0180
Training Iter : 6, Acc : 0.421875, Loss : 1.1789
Training Iter : 7, Acc : 0.40625, Loss : 1.0916
Training Iter : 8, Acc : 0.515625, Loss : 1.0083
Training Iter : 9, Acc : 0.359375, Loss : 1.4293
Training Iter : 10, Acc : 0.359375, Loss : 1.3265
Training Iter : 11, Acc : 0.421875, Loss : 1.1470
Validation Iter : 11, Acc : 0.234375, Loss : 1.4106
Training Iter : 12, Acc : 0.484375, Loss : 1.0661
Training Iter : 13, Acc : 0.296875, Loss : 1.1352
Training Iter : 14, Acc : 0.390625, Loss : 1.0261
Training Iter : 15, Acc : 0.390625, Loss : 1.1072
Training Iter : 16, Acc : 0.5625, Loss : 0.9499
Training Iter : 17, Acc : 0.546875, Loss : 0.8966
Training Iter : 18, Acc : 0.546875, Los

Training Iter : 151, Acc : 0.34375, Loss : 1.1153
Validation Iter : 151, Acc : 0.265625, Loss : 1.1152
Training Iter : 152, Acc : 0.359375, Loss : 1.1143
Training Iter : 153, Acc : 0.359375, Loss : 1.0768
Training Iter : 154, Acc : 0.328125, Loss : 1.1482
Training Iter : 155, Acc : 0.265625, Loss : 1.0960
Training Iter : 156, Acc : 0.328125, Loss : 1.0794
Training Iter : 157, Acc : 0.40625, Loss : 1.1004
Training Iter : 158, Acc : 0.453125, Loss : 1.0568
Training Iter : 159, Acc : 0.46875, Loss : 1.0470
Training Iter : 160, Acc : 0.421875, Loss : 1.0590
Training Iter : 161, Acc : 0.359375, Loss : 1.1440
Validation Iter : 161, Acc : 0.421875, Loss : 1.0890
Training Iter : 162, Acc : 0.25, Loss : 1.1395
Training Iter : 163, Acc : 0.453125, Loss : 1.0962
Training Iter : 164, Acc : 0.359375, Loss : 1.0914
Training Iter : 165, Acc : 0.328125, Loss : 1.1087
Training Iter : 166, Acc : 0.1875, Loss : 1.1119
Training Iter : 167, Acc : 0.203125, Loss : 1.1032
Training Iter : 168, Acc : 0.5625, L

Training Iter : 300, Acc : 0.328125, Loss : 1.1251
Training Iter : 301, Acc : 0.4375, Loss : 1.1030
Validation Iter : 301, Acc : 0.375, Loss : 1.1079
Training Iter : 302, Acc : 0.234375, Loss : 1.1135
Training Iter : 303, Acc : 0.296875, Loss : 1.0921
Training Iter : 304, Acc : 0.296875, Loss : 1.0952
Training Iter : 305, Acc : 0.390625, Loss : 1.1012
Training Iter : 306, Acc : 0.375, Loss : 1.1097
Training Iter : 307, Acc : 0.359375, Loss : 1.0944
Training Iter : 308, Acc : 0.328125, Loss : 1.1148
Training Iter : 309, Acc : 0.390625, Loss : 1.0958
Training Iter : 310, Acc : 0.4375, Loss : 1.0872
Training Iter : 311, Acc : 0.359375, Loss : 1.0877
Validation Iter : 311, Acc : 0.34375, Loss : 1.0997
Training Iter : 312, Acc : 0.359375, Loss : 1.0922
Training Iter : 313, Acc : 0.46875, Loss : 1.0839
Training Iter : 314, Acc : 0.40625, Loss : 1.0801
Training Iter : 315, Acc : 0.5, Loss : 1.0695
Training Iter : 316, Acc : 0.21875, Loss : 1.1242
Training Iter : 317, Acc : 0.4375, Loss : 1.03

Training Iter : 449, Acc : 0.34375, Loss : 1.0880
Training Iter : 450, Acc : 0.234375, Loss : 1.1203
Training Iter : 451, Acc : 0.28125, Loss : 1.0961
Validation Iter : 451, Acc : 0.1875, Loss : 1.1010
Training Iter : 452, Acc : 0.25, Loss : 1.0995
Training Iter : 453, Acc : 0.375, Loss : 1.0986
Training Iter : 454, Acc : 0.40625, Loss : 1.0912
Training Iter : 455, Acc : 0.34375, Loss : 1.0971
Training Iter : 456, Acc : 0.34375, Loss : 1.1062
Training Iter : 457, Acc : 0.265625, Loss : 1.0985
Training Iter : 458, Acc : 0.328125, Loss : 1.0972
Training Iter : 459, Acc : 0.359375, Loss : 1.0985
Training Iter : 460, Acc : 0.390625, Loss : 1.0957
Training Iter : 461, Acc : 0.359375, Loss : 1.0956
Validation Iter : 461, Acc : 0.234375, Loss : 1.1200
Training Iter : 462, Acc : 0.28125, Loss : 1.0977
Training Iter : 463, Acc : 0.28125, Loss : 1.1004
Training Iter : 464, Acc : 0.40625, Loss : 1.0848
Training Iter : 465, Acc : 0.3125, Loss : 1.0944
Training Iter : 466, Acc : 0.359375, Loss : 1.

Training Iter : 597, Acc : 0.296875, Loss : 1.1359
Training Iter : 598, Acc : 0.484375, Loss : 1.0845
Training Iter : 599, Acc : 0.421875, Loss : 1.0494
Training Iter : 600, Acc : 0.359375, Loss : 1.1030
Training Iter : 601, Acc : 0.484375, Loss : 1.0700
Validation Iter : 601, Acc : 0.125, Loss : 1.1247
Training Iter : 602, Acc : 0.328125, Loss : 1.0841
Training Iter : 603, Acc : 0.5, Loss : 1.0571
Training Iter : 604, Acc : 0.4375, Loss : 1.0967
Training Iter : 605, Acc : 0.421875, Loss : 1.1027
Training Iter : 606, Acc : 0.46875, Loss : 1.0963
Training Iter : 607, Acc : 0.484375, Loss : 1.0805
Training Iter : 608, Acc : 0.453125, Loss : 1.0741
Training Iter : 609, Acc : 0.546875, Loss : 0.9991
Training Iter : 610, Acc : 0.34375, Loss : 1.1777
Training Iter : 611, Acc : 0.390625, Loss : 1.0843
Validation Iter : 611, Acc : 0.25, Loss : 1.1743
Training Iter : 612, Acc : 0.296875, Loss : 1.0900
Training Iter : 613, Acc : 0.4375, Loss : 1.0682
Training Iter : 614, Acc : 0.4375, Loss : 1.0

...

Training Iter : 5554, Acc : 0.421875, Loss : 1.0777
Training Iter : 5555, Acc : 0.328125, Loss : 1.0976
Training Iter : 5556, Acc : 0.359375, Loss : 1.0977
Training Iter : 5557, Acc : 0.375, Loss : 1.1212
Training Iter : 5558, Acc : 0.4375, Loss : 1.0838
Training Iter : 5559, Acc : 0.3125, Loss : 1.1293
Training Iter : 5560, Acc : 0.328125, Loss : 1.0954
Training Iter : 5561, Acc : 0.25, Loss : 1.0957
Validation Iter : 5561, Acc : 0.453125, Loss : 1.0972
Training Iter : 5562, Acc : 0.390625, Loss : 1.0995
Training Iter : 5563, Acc : 0.5625, Loss : 1.0746
Training Iter : 5564, Acc : 0.421875, Loss : 1.0771
Training Iter : 5565, Acc : 0.4375, Loss : 1.0696
Training Iter : 5566, Acc : 0.34375, Loss : 1.0968
Training Iter : 5567, Acc : 0.5625, Loss : 1.0545
Training Iter : 5568, Acc : 0.359375, Loss : 1.0861
Training Iter : 5569, Acc : 0.3125, Loss : 1.1185
Training Iter : 5570, Acc : 0.375, Loss : 1.0823
Training Iter : 5571, Acc : 0.46875, Loss : 1.0919
Validation Iter : 5571, Acc : 0.45

Training Iter : 5701, Acc : 0.4375, Loss : 1.0969
Validation Iter : 5701, Acc : 0.328125, Loss : 1.0919
Training Iter : 5702, Acc : 0.265625, Loss : 1.1041
Training Iter : 5703, Acc : 0.25, Loss : 1.1058
Training Iter : 5704, Acc : 0.125, Loss : 1.1108
Training Iter : 5705, Acc : 0.296875, Loss : 1.0974
Training Iter : 5706, Acc : 0.4375, Loss : 1.0900
Training Iter : 5707, Acc : 0.546875, Loss : 1.0828
Training Iter : 5708, Acc : 0.328125, Loss : 1.0990
Training Iter : 5709, Acc : 0.484375, Loss : 1.0903
Training Iter : 5710, Acc : 0.421875, Loss : 1.1007
Training Iter : 5711, Acc : 0.25, Loss : 1.1063
Validation Iter : 5711, Acc : 0.265625, Loss : 1.0991
Training Iter : 5712, Acc : 0.53125, Loss : 1.0894
Training Iter : 5713, Acc : 0.390625, Loss : 1.0867
Training Iter : 5714, Acc : 0.421875, Loss : 1.0973
Training Iter : 5715, Acc : 0.40625, Loss : 1.0917
Training Iter : 5716, Acc : 0.375, Loss : 1.0998
Training Iter : 5717, Acc : 0.390625, Loss : 1.0919
Training Iter : 5718, Acc : 

Training Iter : 5847, Acc : 0.265625, Loss : 1.1232
Training Iter : 5848, Acc : 0.265625, Loss : 1.1355
Training Iter : 5849, Acc : 0.40625, Loss : 1.0886
Training Iter : 5850, Acc : 0.328125, Loss : 1.1242
Training Iter : 5851, Acc : 0.453125, Loss : 1.0836
Validation Iter : 5851, Acc : 0.296875, Loss : 1.1166
Training Iter : 5852, Acc : 0.203125, Loss : 1.1106
Training Iter : 5853, Acc : 0.328125, Loss : 1.0979
Training Iter : 5854, Acc : 0.34375, Loss : 1.0963
Training Iter : 5855, Acc : 0.375, Loss : 1.0949
Training Iter : 5856, Acc : 0.453125, Loss : 1.0702
Training Iter : 5857, Acc : 0.421875, Loss : 1.0919
Training Iter : 5858, Acc : 0.375, Loss : 1.0891
Training Iter : 5859, Acc : 0.46875, Loss : 1.0794
Training Iter : 5860, Acc : 0.390625, Loss : 1.0649
Training Iter : 5861, Acc : 0.4375, Loss : 1.0382
Validation Iter : 5861, Acc : 0.296875, Loss : 1.1263
Training Iter : 5862, Acc : 0.390625, Loss : 1.0495
Training Iter : 5863, Acc : 0.46875, Loss : 1.0257
Training Iter : 5864

Training Iter : 5992, Acc : 0.453125, Loss : 1.0741
Training Iter : 5993, Acc : 0.59375, Loss : 1.0440
Training Iter : 5994, Acc : 0.484375, Loss : 1.0801
Training Iter : 5995, Acc : 0.5, Loss : 1.0585
Training Iter : 5996, Acc : 0.5, Loss : 1.0783
Training Iter : 5997, Acc : 0.578125, Loss : 1.0278
Training Iter : 5998, Acc : 0.578125, Loss : 1.0040
Training Iter : 5999, Acc : 0.28125, Loss : 1.1453


KeyboardInterrupt: 

In [28]:
train_batches

8533

In [27]:
end_time = time.time() - start_time
print("{} seconds".format(end_time))

saver.save(sess, os.path.join(logs_path, 'VGG16_classification_crop', model_name))

3277.8827085494995 seconds


'./input_data\\graph\\VGG16_classification_crop\\vgg16_classification_crop10'

In [29]:
sess.run(infer_iterator.initializer)
y_true, y_pred = sess.run([model.y, model.y_pred], feed_dict={handle:infer_handle})
i = 0

while True:
    try:
        tmp_true, tmp_pred = sess.run([model.y, model.y_pred], feed_dict={handle:infer_handle})
        y_true = np.concatenate((y_true, tmp_true))
        y_pred = np.concatenate((y_pred, tmp_pred))
        if i % 200 == 0:
            print(i)
        i += 1
    except:
        y_true = np.array([np.where(r==1)[0][0] for r in y_true])
        break

0
200
400
600
800
1000
1200
1400
1600
1800
2000


In [43]:
len(np.repeat(label[ts_idx], 10))
# label[ts_idx]


128690

In [37]:
len(label)

64346

In [45]:
y_pred

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [46]:
df2 = pd.DataFrame(data={'y_true':y_true, 'y_pred':y_pred} )
df2.to_csv("true_and_pred.csv", encoding='utf-8', index=False)

In [ ]:
sess.run(infer_iterator.initializer)
y_true, y_pred, y_prob = sess.run([model.y, model.y_pred, model.y_prob], feed_dict={handle:infer_handle})
i = 0

while True:
    try:
        tmp_true, tmp_pred, tmp_prob = sess.run([model.y, model.y_pred, model.y_prob], feed_dict={handle:infer_handle})
        y_true = np.concatenate((y_true, tmp_true))
        y_pred = np.concatenate((y_pred, tmp_pred))
        y_prob = np.concatenate((y_prob, tmp_prob))
        if i % 200 == 0:
            print(i)
        i += 1
    except:
        y_true = np.array([np.where(r==1)[0][0] for r in y_true])
        break

def report_to_df(y_true, y_pred, y_prob):
    cm = skm.confusion_matrix(y_true, y_pred)
    acc = skm.accuracy_score(y_true, y_pred)  # Accuracy
    print("Accuracy : {}".format(acc))
    pd.DataFrame(cm).to_csv("{}_cm.csv".format(model_name), encoding='utf-8')
    
    report = skm.precision_recall_fscore_support(y_true, y_pred)
    out_dict = {
                 "precision" :report[0].round(2)
                ,"recall" : report[1].round(2)
                ,"f1-score" : report[2].round(2)
                ,"support" : report[3]
                }
    out_df = pd.DataFrame(out_dict)
    avg_total = (out_df.apply(lambda x: round(x.mean(), 2) if x.name!="support" else  round(x.sum(), 2)).to_frame().T)
    avg_total.index = ["avg/total"]
    out_df = out_df.append(avg_tot)
    auroc = list()
    for i in range(NUM_CLASSES):
        fpr, tpr, thresholds =skm.roc_curve(y_true, y_prob[:,i], pos_label=i)
        auc = skm.auc(fpr, tpr)
        auroc.append(auc)
    auroc.append(np.mean(auroc))
    auroc_df= pd.DataFrame(auroc, columns=["AUROC"]).set_index([[0,1,2,'avg/total']])
    
    df = pd.concat([out_df, auroc_df], axis=1)
    df.to_csv("{}_report.csv".format(model_name), encoding='utf-8')
    
report_to_df(y_true, y_pred, y_prob)

In [ ]:
report_to_df(y_true, y_pred, y_prob)